In [1]:
!pip install langchain langchain_experimental langchain-google-genai python-dotenv mysql-connector-python pymysql

  Using cached mysql_connector_python-9.0.0-cp312-cp312-win_amd64.whl.metadata (2.0 kB)
Using cached mysql_connector_python-9.0.0-cp312-cp312-win_amd64.whl (14.3 MB)


In [2]:
!pip install tiktoken faiss-cpu sentence-transformers 

  Using cached sentence_transformers-3.0.1-py3-none-any.whl.metadata (10 kB)
  Using cached torch-2.4.0-cp312-cp312-win_amd64.whl.metadata (27 kB)
  Using cached scikit_learn-1.5.1-cp312-cp312-win_amd64.whl.metadata (12 kB)
  Using cached huggingface_hub-0.24.6-py3-none-any.whl.metadata (13 kB)
  Using cached filelock-3.15.4-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2024.6.1-py3-none-any.whl.metadata (11 kB)
  Using cached sympy-1.13.2-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.3-py3-none-any.whl.metadata (5.1 kB)
  Using cached safetensors-0.4.4-cp312-none-win_amd64.whl.metadata (3.9 kB)
  Using cached tokenizers-0.19.1-cp312-none-win_amd64.whl.metadata (6.9 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
   ----- ----------------------

In [3]:
from langchain_google_genai import GoogleGenerativeAI

import os
from dotenv import load_dotenv
load_dotenv() 
GOOGLE_API_KEY="AIxxxxxxxxxxxxxxxxxxxxxxxxxxxOk"
#llm = GoogleGenerativeAI(model="models/text-bison-001", google_api_key=os.environ["GOOGLE_API_KEY"])
llm = GoogleGenerativeAI(model="gemini-pro", google_api_key=GOOGLE_API_KEY)

In [4]:
print(llm("write few lines on Generative AI"))

C:\Users\MANISH\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


Generative AI, a subset of artificial intelligence, enables the creation of original content from scratch. It harnesses algorithms and deep learning models to generate text, images, music, and other forms of media that mimic human creativity. Generative AI has the potential to revolutionize industries such as art, entertainment, and media by automating creative tasks and empowering individuals to express themselves in new and innovative ways.


In [5]:
#from langchain_community.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

from langchain.utilities import SQLDatabase

In [10]:
db_user = "root"
db_password = "manish"
db_host = "localhost"
db_name = "retail_sales_db"

db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",sample_rows_in_table_info=3)


print(db.table_info)


CREATE TABLE sales_tb (
	`TransactionID` INTEGER, 
	`Date` DATE, 
	`CustomerID` VARCHAR(10), 
	`Gender` VARCHAR(10), 
	`Age` INTEGER, 
	`ProductCategory` VARCHAR(50), 
	`Quantity` INTEGER, 
	`PriceperUnit` DECIMAL(10, 2), 
	`TotalAmount` DECIMAL(10, 2)
)ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4

/*
3 rows from sales_tb table:
TransactionID	Date	CustomerID	Gender	Age	ProductCategory	Quantity	PriceperUnit	TotalAmount
1	2023-11-24	CUST001	Male	34	Beauty	3	50.00	150.00
2	2023-02-27	CUST002	Female	26	Clothing	2	500.00	1000.00
3	2023-01-13	CUST003	Male	50	Electronics	1	30.00	30.00
*/


In [11]:
#Convert question to SQL query
from langchain.chains import create_sql_query_chain

chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "How many customers are there"})
response

'```sql\nSELECT COUNT(DISTINCT `CustomerID`) AS `TotalCustomers`\nFROM sales_tb;\n```'

In [12]:
cleaned_query = response.strip('```sql\n').strip('\n```')
print(cleaned_query)

SELECT COUNT(DISTINCT `CustomerID`) AS `TotalCustomers`
FROM sales_tb;


In [13]:
# Execute the cleaned query
result = db.run(cleaned_query)
print(result)

[(29,)]


In [14]:
chain = create_sql_query_chain(llm, db)
def execute_query(question):
    try:
        # Generate SQL query from question
        response = chain.invoke({"question": question})
        print(response)
        print("###################################################")
        # Strip the formatting markers from the response
        cleaned_query = response.strip('```sql\n').strip('\n```')
        print(cleaned_query)
        print("###################################################")        
        # Execute the cleaned query
        result = db.run(cleaned_query)
        print("###################################################")        
        # Display the result
        print(result)
    except ProgrammingError as e:
        print(f"An error occurred: {e}")

In [15]:
q1 = "How many unique customers are there for each product category"
execute_query(q1)

```sql
SELECT 
    `ProductCategory`, 
    COUNT(DISTINCT `CustomerID`) AS `Number of Customers` 
FROM 
    `sales_tb` 
GROUP BY 
    `ProductCategory` 
ORDER BY 
    `Number of Customers` DESC 
LIMIT 
    5;
```
###################################################
SELECT 
    `ProductCategory`, 
    COUNT(DISTINCT `CustomerID`) AS `Number of Customers` 
FROM 
    `sales_tb` 
GROUP BY 
    `ProductCategory` 
ORDER BY 
    `Number of Customers` DESC 
LIMIT 
    5;
###################################################
###################################################
[('Clothing', 13), ('Beauty', 8), ('Electronics', 8)]


In [16]:
q2 = "Calculate total sales amount per product category:"
execute_query(q2)

```sql
SELECT 
  `ProductCategory`, SUM(`TotalAmount`) AS `TotalSales` 
FROM 
  `sales_tb` 
GROUP BY 
  `ProductCategory` 
LIMIT 
  5;
```
###################################################
SELECT 
  `ProductCategory`, SUM(`TotalAmount`) AS `TotalSales` 
FROM 
  `sales_tb` 
GROUP BY 
  `ProductCategory` 
LIMIT 
  5;
###################################################
###################################################
[('Beauty', Decimal('1455.00')), ('Clothing', Decimal('5040.00')), ('Electronics', Decimal('5310.00'))]


In [17]:
q3 = "calculates the average age of customers grouped by gender."
execute_query(q3)

```sql
SELECT `Gender`, AVG(`Age`) AS `AverageAge` 
FROM `sales_tb`
GROUP BY `Gender`
LIMIT 5;
```
###################################################
SELECT `Gender`, AVG(`Age`) AS `AverageAge` 
FROM `sales_tb`
GROUP BY `Gender`
LIMIT 5;
###################################################
###################################################
[('Male', Decimal('35.2143')), ('Female', Decimal('43.3333'))]


In [18]:
q4 = "identify the top spending customers based on their total amount spent."
execute_query(q4)

```sql
SELECT 
    `CustomerID`, 
    SUM(`TotalAmount`) AS `TotalSpent`
FROM 
    `sales_tb`
GROUP BY 
    `CustomerID`
ORDER BY 
    `TotalSpent` DESC
LIMIT 
    5;
```
###################################################
SELECT 
    `CustomerID`, 
    SUM(`TotalAmount`) AS `TotalSpent`
FROM 
    `sales_tb`
GROUP BY 
    `CustomerID`
ORDER BY 
    `TotalSpent` DESC
LIMIT 
    5;
###################################################
###################################################
[('CUST015', Decimal('2000.00')), ('CUST013', Decimal('1500.00')), ('CUST016', Decimal('1500.00')), ('CUST002', Decimal('1000.00')), ('CUST026', Decimal('1000.00'))]


In [19]:
q5 = "counts the number of transactions made each month."
execute_query(q5)

```sql
SELECT
    COUNT(*) AS `Total Transactions`
FROM
    sales_tb
GROUP BY
    STRFTIME('%Y-%m', `Date`)
ORDER BY
    `Date` DESC
LIMIT 5;
```
###################################################
SELECT
    COUNT(*) AS `Total Transactions`
FROM
    sales_tb
GROUP BY
    STRFTIME('%Y-%m', `Date`)
ORDER BY
    `Date` DESC
LIMIT 5;
###################################################


NameError: name 'ProgrammingError' is not defined